# Betaseries extraction

This script combines calculated whole brain trial beta-maps with brain parcellation and extracts trial beta-series for predefined set of brain regions. This analysis step has to be conducted separately for each parcellation. `NiftiSpheresMasker` is used for signal extraction with parameters:
- `allow_overlap=False`: ensures that parcellation has no everlapping spheres
- `standardize=True`: z-scores signal along trials dimension
- `detrend=False`: disable detrending since we are no longer in a time domain
- `high_pass=None`: disable high-pass filtering
- `low_pass=None`: disable low-pass filtering

Use of brain mask can be enabled / disabled by setting `use_mask` flag. Note that due to signal dropout in orbitofrontal regions, that will likely lead to an error raised by masker (some ROIs will fall out of the mask). Therefore by default `use_mask=False`. Spheres outside of the brain mask – i.e. these without signal, will be removed in the next analysis step during network construction. 

Here, data from all subject and both task conditions are also aggregated into single `.npy` file for convenience. Output file is stored as:

> `betaseries/<atlas_name>/betaseries_aggregated.npy`

Output array has shape `n_subjects` x `n_conditions` x `n_trials` x `n_rois`. Metadata corresponding to first thee dimensions can be found in `behavioral_data_clean_all.json` file.  

In [ ]:
import json
import os
from os.path import join
from pathlib import Path

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
from dn_utils.behavioral_models import load_behavioral_data
from nilearn.input_data import NiftiSpheresMasker

In [ ]:
path_root = os.environ.get('DECIDENET_PATH')

path_derivatives = join(path_root, 'data/main_fmri_study/derivatives') 
path_sourcedata = join(path_root, 'data/main_fmri_study/sourcedata') 

path_beh = join(path_sourcedata, 'behavioral')
path_bsc = join(path_derivatives, 'bsc')
path_parcellations = join(path_derivatives, 'parcellations')
path_data_paths = join(path_derivatives, 'data_paths')

path_betamaps = join(path_bsc, 'betamaps')
path_betaseries = join(path_bsc, 'betaseries')

Path(path_betaseries).mkdir(exist_ok=True)

### Select brain parcellation & masker options

In [ ]:
atlas = 'combined_roi'
roi_table_fname = 'combined_roi_table.csv'

# Arguments for NiftiSpheresMasker
masker_kwargs = {
    'allow_overlap': False, 
    'standardize': True, 
    'detrend': False, 
    'high_pass': None,
    'low_pass': None
}

# Whether to use individual brain mask during signal extraction
use_mask = False

In [ ]:
# Load ROI data
df_roi = pd.read_csv(join(path_parcellations, atlas, roi_table_fname))

# Load behavioral data
beh, meta = load_behavioral_data(path_beh, verbose=False)
n_subjects = beh.shape[0]
n_conditions = beh.shape[1]
n_trials = beh.shape[2]

# Load masks
with open(join(path_data_paths, 'mask_filenames.json'), 'r') as f:
    mask_files = json.loads(f.read())

# Load betamats
imgs = {'prlrew': [], 'prlpun': []}
for con_idx, con in enumerate(meta['dim2']):
    for sub_idx, sub in enumerate(meta['dim1']):
        img_fname = f'sub-{sub}_task-prl{con}_betamaps.nii.gz' 
        imgs[f'prl{con}'].append(nib.load(join(path_betamaps, img_fname)))
        
df_roi.head()

In [ ]:
n_rois = len(df_roi)
seeds = [tuple(coords[1]) for coords in df_roi[['x', 'y', 'z']].iterrows()]

maskers = {}

for radius in df_roi['radius(mm)'].unique():
    # Select ROIs with given radius
    roi_indices = np.flatnonzero(df_roi['radius(mm)'] == radius)
    
    # Create masker for single radius value
    masker = NiftiSpheresMasker(
        [seeds[idx] for idx in roi_indices], 
        radius=radius,                
        mask_img=None,
        **masker_kwargs
    )
    
    maskers[radius] = {'masker': masker, 'indices': roi_indices}

In [ ]:
betaseries_aggregated = np.zeros((n_subjects, n_conditions, n_trials, n_rois))

for sub_idx, sub in enumerate(meta['dim1']):
    for con_idx, con in enumerate(meta['dim2']):
        print(f'sub {sub} con {con}')
        mask_img = nib.load(mask_files[f'prl{con}'][sub_idx])
        beta_img = imgs[f'prl{con}'][sub_idx]

        betaseries = np.zeros((n_trials, n_rois))

        for radius in maskers:
            # Get indices to insert computed timeseries into right positions
            roi_indices = maskers[radius]['indices']

            # Get right spheres masker and add right whole-brain mask 
            masker = maskers[radius]['masker']
            
            # Apply mask
            if use_mask:
                masker.mask_img = mask_img

            # Extract timeseries          
            betaseries[:, roi_indices] = masker.fit_transform(beta_img)
                
        betaseries_aggregated[sub_idx, con_idx] = betaseries
        
# Store betaseries
Path(join(path_betaseries, atlas)).mkdir(exist_ok=True, parents=True)
np.save(join(path_betaseries, atlas, 'betaseries_aggregated.npy'), 
        betaseries_aggregated)